# Generating the N-Gram Models

An N-Gram model is simply an enumeration of n-pairs of words, their frequency and probability. The ngrams will be represented in a csv spreadsheet. This is so they are compatible with Pandas and we do not have to recompute them each time we wish to use them.

There are 3 models in total: Vanilla, Laplace and UNK. The Laplace model will include an occurence of every possible combination of n words known in the corpus. Saving such a model would be slow and wasteful. During text generation and perplexity calculation the laplace model will make use of the vanilla model and if an unkown sequence of words is encountered, then we perform laplace smoothing.

In [1]:
import os
import pandas as pd
import seaborn as sns

In [2]:
folder = os.path.join(os.getcwd(),'..','data','korpus')

vanilla = os.path.join(os.getcwd(),'..','data','korpus','ngram','vanilla')
laplace = os.path.join(os.getcwd(),'..','data','korpus','ngram','laplace')
unk = os.path.join(os.getcwd(),'..','data','korpus','ngram','unk')

### Generating Vanilla Language Model

The vanilla models will be implemented by first counting the frequency of the ngrams (sequence of n words) within the korpus. Then the probabiltiies will be calculated.

Bigrams and Trigrams make use of the previous ngram (Unigram and Bigram respectively). Hence when the unigram or bigram is generated, a dictionary is created by using Pandas df.to_dict. This is so to facilitate O(1) access time when searching.

In [3]:
df = pd.read_csv(os.path.join(folder,'korpus_clean.csv'))

word_count = pd.read_csv(os.path.join(folder,'korpus_frequency.csv'))['Frequency'].sum()
words = df['Word'].to_numpy()


#### Unigram

In [4]:
%%time

#Calculating frequencies

def calcualte_unigram(words, word_count: int):
    unigram = {}
    
    #Calculating frequency.
    for i in range(len(words)):
        first  = words[i]

        if f'{first}' in unigram:
            unigram[f'{first}'] += 1
        else:
            unigram[f'{first}'] = 1
            
    df_unigram = pd.DataFrame(unigram.items(), columns=['Unigram', 'Frequency'])
    
    #Calculating probability.
    df_unigram['Probability'] = [freq/word_count for freq in unigram.values()]
    df_unigram.dropna()
    print('Finished!')
    return df_unigram


df_unigram = calcualte_unigram(words, word_count)

Finished!
Wall time: 7.56 s


In [5]:
%%time
#Saving unigram
dict_unigram = df_unigram.set_index('Unigram').T.to_dict('list')
df_unigram.to_csv(os.path.join(vanilla,'unigram.csv'), index=False)

Wall time: 7.25 s


#### Bigram

In [6]:
%%time
#Bigram
def calculate_bigram(words, unigram: dict): 
    bigram = {}

    #Calculate frequency.
    for i in range(len(words)-1):
        first  = words[i]
        second = words[i+1]
        
        if f'{first};{second}' in bigram:
            bigram[f'{first};{second}'] += 1
        else:
            bigram[f'{first};{second}'] = 1        
    
    df_bigram = pd.DataFrame(bigram.items(), columns=['Bigram', 'Frequency'])
    
    df_bigram['Probability'] = [bi_freq/unigram[bi.split(';')[0]][0] for bi,bi_freq in bigram.items()]
    
    print('Finished!')
    return df_bigram
    
df_bigram = calculate_bigram(words, dict_unigram)


Finished!
Wall time: 23.5 s


In [7]:
df_bigram

,Bigram,Frequency,Probability
0,<s>;L-,91248,8.834669e-02
1,L-;għan,832,7.575965e-03
2,għan;prinċipali,57,4.983389e-03
3,prinċipali;ta',583,1.349537e-01
4,ta';Conectando,1,9.329498e-07
...,...,...,...
2942958,Kwalifika;għall-,1,6.666667e-02
2942959,Biex;jikkwaliflkaw,1,3.586801e-04
2942960,jikkwaliflkaw;għall-,1,1.000000e+00
2942961,għandhom;jinkishu,1,7.877179e-06


In [8]:
#Saving bigram

dict_bigram = df_bigram.set_index('Bigram').T.to_dict('list')
df_bigram.to_csv(os.path.join(vanilla,'bigram.csv'), index=False)

#### Trigram

In [9]:
%%time

def calculate_trigram(words, bigram: dict): 
    trigram = {}

    #Calculate frequency.
    for i in range(len(words)-2):
        first  = words[i]
        second = words[i+1]
        third = words[i+2]

        if f'{first};{second};{third}' in trigram:
            trigram[f'{first};{second};{third}'] += 1
        else:
            trigram[f'{first};{second};{third}'] = 1
            
            
    df_trigram = pd.DataFrame(trigram.items(), columns=['Trigram', 'Frequency'])
    
    #Calculate probability.
    probability = []

    for tri,tri_freq in trigram.items():
        first,second = tri.split(';')[:2]
        bi_freq = dict_bigram[f'{first};{second}'][0]
        probability.append(tri_freq/bi_freq)

    df_trigram['Probability'] = probability
    
    print('Finished!')
    return df_trigram

df_trigram = calculate_trigram(words, dict_bigram)

Finished!
Wall time: 50.5 s


In [10]:
df_trigram

,Trigram,Frequency,Probability
0,<s>;L-;għan,779,0.008537
1,L-;għan;prinċipali,24,0.028846
2,għan;prinċipali;ta',14,0.245614
3,prinċipali;ta';Conectando,1,0.001715
4,ta';Conectando;Mundos,1,1.000000
...,...,...,...
8614064,jinkishu;għat-;tliet,1,1.000000
8614065,għat-;tliet;taqsimiet,1,0.009901
8614066,tliet;taqsimiet;studju,1,0.071429
8614067,anqas;4;kredti,1,0.125000


In [11]:
#Saving trigram
df_trigram.to_csv(os.path.join(vanilla,'trigram.csv'), index=False)

### Generate UNK Model

To generate the UNK model, first we remove all words in the vanilla unigram model that have a frequency of 2 or less. We create an extra token called \<UNK> that will represent all the removed words. The frequency of the \<UNK> willbe the sum of the frequency of the removed words.

#### Unigram

In [12]:
#Sum frequencies for all word with frequency less than 3.

#Load vanilla unigram.
df_unigram = pd.read_csv(os.path.join(vanilla,'unigram.csv'))

#Calculate Frequency and Probability.
condition = df_unigram['Frequency'] < 3

unk_frequency = df_unigram[condition].Frequency.sum()
unk_probability = unk_frequency/word_count

#Remove the words that occure less than 3 times.
df_unigram = df_unigram.drop(df_unigram[condition].index)

#Add the <UNK> token.
df_unk = pd.DataFrame({'Unigram': '<UNK>', 'Frequency': unk_frequency, 'Probability': unk_probability}
                   ,index=[0])
df_unigram = pd.concat([df_unk,df_unigram], ignore_index = True)

df_unigram = df_unigram.dropna()

#Save model.
df_unigram.to_csv(os.path.join(unk,'unigram.csv'), index=False)
dict_unigram = df_unigram.set_index('Unigram').T.to_dict('list')

#Replace low frequency words with <UNK>

for i in range(len(words)):
    #If the word is not in dict_unigram then it means it was removed and set to <UNK>
    if words[i] not in dict_unigram:
        words[i] = '<UNK>'


df_unigram

,Unigram,Frequency,Probability
0,<UNK>,211612,8.029837e-03
1,<s>,1032840,3.919218e-02
2,L-,109821,4.167272e-03
3,għan,11438,4.340268e-04
4,prinċipali,4320,1.639269e-04
...,...,...,...
108415,231.41,5,1.897302e-07
108416,irritar,3,1.138381e-07
108417,mrara,3,1.138381e-07
108418,sapuri,3,1.138381e-07


Then we recalculate the Bigram and Trigram based on this \<UNK> unigram

#### Bigram

In [13]:
df_bigram = calculate_bigram(words, dict_unigram)

Finished!


In [14]:
df_bigram.to_csv(os.path.join(unk,'bigram.csv'), index=False)
dict_bigram = df_bigram.set_index('Bigram').T.to_dict('list')

df_bigram

,Bigram,Frequency,Probability
0,<s>;L-,91248,8.834669e-02
1,L-;għan,832,7.575965e-03
2,għan;prinċipali,57,4.983389e-03
3,prinċipali;ta',583,1.349537e-01
4,ta';Conectando,1,9.329498e-07
...,...,...,...
2612604,Klassi;fl-,1,9.541985e-04
2612605,kors;studju,1,3.588088e-04
2612606,327.193;ħ,1,3.333333e-01
2612607,ECON;it-,1,1.428571e-01


#### Trigram

In [15]:
%%time

df_trigram = calculate_trigram(words, dict_bigram)

Finished!
Wall time: 54.8 s


In [16]:
df_trigram.to_csv(os.path.join(unk,'trigram.csv'), index=False)

df_trigram

,Trigram,Frequency,Probability
0,<s>;L-;għan,779,0.008537
1,L-;għan;prinċipali,24,0.028846
2,għan;prinċipali;ta',14,0.245614
3,prinċipali;ta';Conectando,1,0.001715
4,ta';Conectando;Mundos,1,1.000000
...,...,...,...
8332346,kredtu;għandhom;<UNK>,1,0.076923
8332347,għat-;tliet;taqsimiet,1,0.009901
8332348,tliet;taqsimiet;studju,1,0.071429
8332349,anqas;4;kredti,1,0.125000
